# BeautifulSoup Scraping

In [1]:
# import jtplot module in notebook
from jupyterthemes import jtplot

# choose which theme to inherit plotting style from
# onedork | grade3 | oceans16 | chesterish | monokai | solarizedl | solarizedd
jtplot.style(theme='onedork')

In [2]:
# import the libraries
import requests
import csv
from bs4 import BeautifulSoup
import re
from robobrowser import RoboBrowser
import pandas as pd

<h2>Structure</h2>
<ul>
    <li>We can't access product reviews unless we're logged in as a registered user. So we have to handle login.</li>
    <li>We will be using the most reviewed products page</li>
    <li>We will focus on "Foundation" products</li>
    <li>There are 7 different "Foundation" subcategories with its own unique "CategoryID" in its url
        <ol>
            <li><strong>Powder</strong> - CategoryID: 503</li>
            <li><strong>Primer/Corrector</strong> - CategoryID: 504</li>
            <li><strong>Stick</strong> - CategoryID: 506</li>
            <li><strong>Tinted Moisturizer</strong> - CategoryID: 505</li>
            <li><strong>Liquid</strong> - CategoryID: 502</li>
            <li><strong>Crème</strong> - CategoryID: 501</li>
            <li><strong>BB Cream</strong> - CategoryID: 507</li>
        </ol>
    </li>
    <li>For each subcategory there are products that each have
        <ul>
            <li><strong>Brand Name</strong> </li>
            <li><strong>Product Name</strong> </li>
            <li><strong>Category</strong> </li>
            <li><strong>Average Rating</strong> </li>
            <li><strong>Total Number of Reviews</strong> </li>
            <li><strong>% That Would Buy Again</strong></li>
        </ul>
    </li>
    <li>We will be scraping the first 5 pages of products for each subcategory, with the exception of "Stick" as "Stick" only has 2 pages of products, and so we will only scrape two pages of products for "Stick"</li>
    <li>When we click on a product page there are reviews on the bottom. For each review we will scrape
        <ul>
            <li><strong>Rating</strong> - 1 to 5 lipsticks (aka stars)</li>
            <li><strong>username</strong> </li>
            <li><strong>date</strong> - date that the review was posted</li>
            <li><strong>age</strong> - age range of the reviewer</li>
            <li><strong>skin</strong> - skin information of the reviewer</li>
            <li><strong>hair</strong> - hair information of the reviewer</li>
            <li><strong>eyes</strong> - eye color of the reviewer</li>
            <li><strong>review</strong> - the review itself</li>
        </ul>
    </li>
</ul>


### Login

In [3]:
# login_url
login_url = "https://www.makeupalley.com/account/login.asp"

In [4]:
# let's login
browser = RoboBrowser(history=True)
browser.open(login_url)
form = browser.get_form(action='/account/login.asp')

form["UserName"] = input("Enter Username: ")
form["Password"] = input("Enter Password: ")
browser.session.headers['Referer'] = login_url

browser.submit_form(form)
# print(str(browser.select))

print("Login Successful!")

c:\python\python37\lib\site-packages\robobrowser\browser.py:40: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 40 of the file c:\python\python37\lib\site-packages\robobrowser\browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  features=self.browser.parser,


Enter Username: ugca
Enter Password: barua123
Login Successful!


### Begin Scraper

In [5]:
# Store the different categories in a list
categories = [
    "Powder",
    "Primer/Corrector",
    "Stick",
    "Tinted Moisturizer",
    "Liquid",
    "Crème",
    "BB Cream"
]

In [6]:
# Create a dictionary to to match each Subcategory with its CategoryID
cat_id = dict()
cat_id["Powder"] = 503
cat_id["Primer/Corrector"] = 504
cat_id["Stick"] = 506
cat_id["Tinted Moisturizer"] = 505
cat_id["Liquid"] = 502
cat_id["Crème"] = 501
cat_id["BB Cream"] = 507

In [7]:
# store product info: brand_name, product_name, category, average_rating, total_reviews, buy_again_percentage%
product_info = []

In [8]:
# store output for each review: product_name, rating, username, date, age, skin, hair, eyes, review
reviews = []

In [9]:
# skeleton url
base_url = "https://www.makeupalley.com/product/browse.asp/page=/pagesize=15/CategoryId=/topten=reviewed/AgeRange=0/"
base = "https://www.makeupalley.com"

# iterate through each category of foundation
for cat in categories:
    # 2 pages of products for stick
    if(cat == "Stick"):
        for i in range(1,3):
            # adjust the urls
            product_url = base_url.replace("page=", "page=" + str(i))
            product_url = product_url.replace("CategoryId=", "CategoryId=" + str(cat_id[cat]))
            
            # collect the url with the requests library
            page = requests.get(product_url)
            print(product_url)
            
            # get the html of the page in string form
            page_html = page.text

            # create the BeautifulSoup object that takes in the html in str form and a html/xml parser of choice either html.parser or lxml
            soup = BeautifulSoup(page_html, 'html.parser')

            # Pull all text from the div class : search-results
            product_search_results = soup.find(class_ = "search-results")
            
            # Grab all the rows of products
            product_list = product_search_results.find_all("tr")
            
            # get the attributes
            for product in product_list:
                # store product data
                row_product_data = []
                
                # grab all column information
                columns = product.find_all("td")
                
                # store variable for the product link
                product_link = ""
                for col in range(len(columns)):
                    # first column is brand
                    if(col == 0):
                        row_product_data.append(columns[col].text)
                        #print(columns[col].text)
                    
                    # second column is the product
                    elif(col == 1):
                        product_name = columns[col].find_all("a", href = True)[1]
                        row_product_data.append(product_name.text)
                        product_link = base + product_name["href"]
                        #print(product_name.text)
                       # print(product_link)
                    
                    # third column is the category name
                    elif(col == 2):
                        the_cat = columns[col].text 
                        row_product_data.append(the_cat)
                       # print(the_cat)
                        
                    # fourth column is avg rating
                    elif(col == 3):
                        avg_rating = float(columns[col].text)
                        row_product_data.append(avg_rating)
                        #print(avg_rating)
                    
                    # fifth columns is number of reviews
                    elif(col == 4):
                        num_reviews = int(columns[col].text.replace(",", ""))
                        row_product_data.append(num_reviews)
                        #print(num_reviews)
                        
                    # sixth column is buy again percentage
                    elif(col == 5):
                        buy_again = float(columns[col].text.replace("%", "")) / 100.0
                        row_product_data.append(buy_again)
                        #print(buy_again)
                        
                # throw this row data into the product_info list
                if(len(row_product_data) != 0):
                    product_info.append(row_product_data)
                    
                    # lets go to the product link
                    #print("This the product link")
                    #print(product_link)
                    product_page = requests.get(product_link)
                    
                    # get the html of the page in string form
                    product_page_html = product_page.text
                    
                    # create the BeautifulSoup object that takes in the html in str form and a html/xml parser of choice either html.parser or lxml
                    soup2 = BeautifulSoup(product_page_html, 'html.parser')
                    
                    # first find the product_id 
                    product_id_search = soup2.find("div" , {"id":"ItemId"})
                    try:
                        product_id = product_id_search.text
                        print("product_id:", product_id, "Scraping")
                        # grab the last page of reviews which is the href of the second to last div of class = track_paging_
                        page_trackers = soup2.find_all(class_ = "track_Paging_", href = True)
                        last_page = page_trackers[-2]["href"]
                    except:
                        print("this shit failed")
                        continue
                         
                    # do something hacky/ string manipulations to extract last page number
                    last_page_number = last_page[:-1]
                    start_index = last_page_number.find("page=")
                    last_page_number = int(last_page_number[start_index + 5:])
                    
                    #print("last_page:", last_page)
                    #print("last page number:", last_page_number)
                    
                    # skeleton
                    base_product_url = "https://www.makeupalley.com/product/showreview.asp/ItemID=/page=/"
    
                    # go through all the pages to scrape reviews!
                    # last_page_number + 1
                    for i in range(1, last_page_number + 1):
                        # update the product url
                        new_product_url = base_product_url.replace("ItemID=","ItemID=" + product_id)
                        new_product_url = new_product_url.replace("page=", "page="+ str(i))
                        #print(new_product_url)
                        
                        # get the html of the page in string form
                        new_product_page = requests.get(new_product_url)
                        new_product_page_html = new_product_page.text
                        
                        # create a beautiful soup object
                        soup3 = BeautifulSoup(new_product_page_html, 'html.parser')
                        
                        comment_list = soup3.find(id = "reviews-wrapper")
                        
                        real_comment_list = comment_list.find_all(class_ = "comments")
                        
                        # iterate through each comment
                        for comment in real_comment_list:
                            # store row data
                            row = []
                            
                            # get the product type
                            row.append(cat)
                            
                            # get the productname
                            row.append(product_name.text)
                            
                            # get the ratings
                            rating = comment.find(class_ = "lipies")
                            score = rating.find("span")
                            row.append(score["class"][0][2])
                            
                            # get username
                            username = comment.find(class_ = "user-name")
                            row.append(username.text.replace("\t", ""))
                            
                            # get the date
                            date = comment.find(class_ = "date")
                            row.append(date.text)
                            
                            # get age, skin, hair, and eyes
                            traits = comment.find(class_ = "important")
                            clean_traits = traits.text.replace("Age:", "")
                            clean_traits = clean_traits.replace("Skin", "")
                            clean_traits = clean_traits.replace("Hair", "")
                            clean_traits = clean_traits.replace("Eyes", "")
                            clean_traits = clean_traits.split(":")
                            for k in range(len(clean_traits)):
                                row.append(clean_traits[k].strip())
                            
                            # get the review
                            review = comment.find(class_ = "break-word")
                            try:
                                row.append(review.text.replace("\t",""))
                            except:
                                try:
                                    review = comment.find(class_ = "1break-word")
                                    row.append(review.text.replace("\t", ""))
                                except:
                                    print("it didn't work")
                                    continue
                            
                            # throw the row of data into reviews
                            reviews.append(row)
                    print("product_id:", product_id, "Complete")
                    
    # 5 pages of products for the rest
    else:
        for i in range(1,6):
            # adjust the urls
            product_url = base_url.replace("page=", "page=" + str(i))
            product_url = product_url.replace("CategoryId=", "CategoryId=" + str(cat_id[cat]))
            
            # collect the url with the requests library
            page = requests.get(product_url)
            print(product_url)
            
            # get the html of the page in string form
            page_html = page.text

            # create the BeautifulSoup object that takes in the html in str form and a html/xml parser of choice either html.parser or lxml
            soup = BeautifulSoup(page_html, 'html.parser')

            # Pull all text from the div class : search-results
            product_search_results = soup.find(class_ = "search-results")
            
            # Grab all the rows of products
            product_list = product_search_results.find_all("tr")
            
            # get the attributes
            for product in product_list:
                # store product data
                row_product_data = []
                
                # grab all column information
                columns = product.find_all("td")
                
                # store variable for the product link
                product_link = ""
                for col in range(len(columns)):
                    # first column is brand
                    if(col == 0):
                        row_product_data.append(columns[col].text)
                        #print(columns[col].text)
                    
                    # second column is the product
                    elif(col == 1):
                        product_name = columns[col].find_all("a", href = True)[1]
                        row_product_data.append(product_name.text)
                        product_link = base + product_name["href"]
                        #print(product_name.text)
                       # print(product_link)
                    
                    # third column is the category name
                    elif(col == 2):
                        the_cat = columns[col].text 
                        row_product_data.append(the_cat)
                       # print(the_cat)
                        
                    # fourth column is avg rating
                    elif(col == 3):
                        avg_rating = float(columns[col].text)
                        row_product_data.append(avg_rating)
                        #print(avg_rating)
                    
                    # fifth columns is number of reviews
                    elif(col == 4):
                        num_reviews = int(columns[col].text.replace(",", ""))
                        row_product_data.append(num_reviews)
                        #print(num_reviews)
                        
                    # sixth column is buy again percentage
                    elif(col == 5):
                        buy_again = float(columns[col].text.replace("%", "")) / 100.0
                        row_product_data.append(buy_again)
                        #print(buy_again)
                        
                # throw this row data into the product_info list
                if(len(row_product_data) != 0):
                    product_info.append(row_product_data)
                    
                    # lets go to the product link
                    #print("This the product link")
                    #print(product_link)
                    product_page = requests.get(product_link)
                    
                    # get the html of the page in string form
                    product_page_html = product_page.text
                    
                    # create the BeautifulSoup object that takes in the html in str form and a html/xml parser of choice either html.parser or lxml
                    soup2 = BeautifulSoup(product_page_html, 'html.parser')
                    
                    # first find the product_id 
                    product_id_search = soup2.find("div" , {"id":"ItemId"})
                    try:
                        product_id = product_id_search.text
                        print("product_id:", product_id, "Scraping")
                        page_trackers = soup2.find_all(class_ = "track_Paging_", href = True)
                        last_page = page_trackers[-2]["href"]
                    except:
                        print("this shit failed")
                        continue
                    
                    # do something hacky/ string manipulations to extract last page number
                    last_page_number = last_page[:-1]
                    start_index = last_page_number.find("page=")
                    last_page_number = int(last_page_number[start_index + 5:])
                    
                    #print("last_page:", last_page)
                    #print("last page number:", last_page_number)
                    
                    # skeleton
                    base_product_url = "https://www.makeupalley.com/product/showreview.asp/ItemID=/page=/"
    
                    # go through all the pages to scrape reviews!
                    # last_page_number + 1
                    for i in range(1, last_page_number + 1):
                        # update the product url
                        new_product_url = base_product_url.replace("ItemID=","ItemID=" + product_id)
                        new_product_url = new_product_url.replace("page=", "page="+ str(i))
                        #print(new_product_url)
                        
                        # get the html of the page in string form
                        new_product_page = requests.get(new_product_url)
                        new_product_page_html = new_product_page.text
                        
                        # create a beautiful soup object
                        soup3 = BeautifulSoup(new_product_page_html, 'html.parser')
                        try:
                            comment_list = soup3.find(id = "reviews-wrapper")
                            real_comment_list = comment_list.find_all(class_ = "comments")
                        except:
                            print("Skipping a page this didn't work")
                            continue
                        
                        # iterate through each comment
                        for comment in real_comment_list:
                            # store row data
                            row = []
                            
                            # get the product type
                            row.append(cat)
                            
                            # get the productname
                            row.append(product_name.text)
                            
                            # get the ratings
                            rating = comment.find(class_ = "lipies")
                            score = rating.find("span")
                            row.append(score["class"][0][2])
                            
                            # get username
                            username = comment.find(class_ = "user-name")
                            row.append(username.text.replace("\t", ""))
                            
                            # get the date
                            date = comment.find(class_ = "date")
                            row.append(date.text)
                            
                            # get age, skin, hair, and eyes
                            traits = comment.find(class_ = "important")
                            clean_traits = traits.text.replace("Age:", "")
                            clean_traits = clean_traits.replace("Skin", "")
                            clean_traits = clean_traits.replace("Hair", "")
                            clean_traits = clean_traits.replace("Eyes", "")
                            clean_traits = clean_traits.split(":")
                            for k in range(len(clean_traits)):
                                row.append(clean_traits[k].strip())
                            
                            # get the review
                            review = comment.find(class_ = "break-word")
                            try:
                                row.append(review.text.replace("\t",""))
                            except:
                                try:
                                    review = comment.find(class_ = "1break-word")
                                    row.append(review.text.replace("\t", ""))
                                except:
                                    print("it didn't work")
                                    continue
                            
                            # throw the row of data into reviews
                            reviews.append(row)
                            
                    print("product_id:", product_id, "Complete")
                        
                        
                        
    

https://www.makeupalley.com/product/browse.asp/page=1/pagesize=15/CategoryId=503/topten=reviewed/AgeRange=0/
product_id: 9246 Scraping
product_id: 9246 Complete
product_id: 25754 Scraping
product_id: 25754 Complete
product_id: 15111 Scraping
product_id: 15111 Complete
product_id: 78607 Scraping
product_id: 78607 Complete
product_id: 74290 Scraping
product_id: 74290 Complete
product_id: 120907 Scraping
product_id: 120907 Complete
product_id: 13145 Scraping
product_id: 13145 Complete
product_id: 100359 Scraping
product_id: 100359 Complete
product_id: 89705 Scraping
product_id: 89705 Complete
product_id: 67857 Scraping
product_id: 67857 Complete
product_id: 102364 Scraping
product_id: 102364 Complete
product_id: 70540 Scraping
product_id: 70540 Complete
product_id: 11317 Scraping
product_id: 11317 Complete
product_id: 116379 Scraping
product_id: 116379 Complete
product_id: 921 Scraping
product_id: 921 Complete
https://www.makeupalley.com/product/browse.asp/page=2/pagesize=15/CategoryId=50

product_id: 131001 Scraping
product_id: 131001 Complete
product_id: 93119 Scraping
product_id: 93119 Complete
https://www.makeupalley.com/product/browse.asp/page=5/pagesize=15/CategoryId=504/topten=reviewed/AgeRange=0/
product_id: 149634 Scraping
product_id: 149634 Complete
product_id: 153833 Scraping
product_id: 153833 Complete
product_id: 117804 Scraping
product_id: 117804 Complete
product_id: 148155 Scraping
product_id: 148155 Complete
product_id: 39860 Scraping
product_id: 39860 Complete
product_id: 11310 Scraping
product_id: 11310 Complete
product_id: 4780 Scraping
product_id: 4780 Complete
product_id: 79093 Scraping
product_id: 79093 Complete
product_id: 153415 Scraping
product_id: 153415 Complete
product_id: 44695 Scraping
product_id: 44695 Complete
product_id: 91049 Scraping
product_id: 91049 Complete
product_id: 135831 Scraping
product_id: 135831 Complete
product_id: 66429 Scraping
product_id: 66429 Complete
product_id: 154387 Scraping
product_id: 154387 Complete
product_id: 9

https://www.makeupalley.com/product/browse.asp/page=2/pagesize=15/CategoryId=502/topten=reviewed/AgeRange=0/
product_id: 176426 Scraping
product_id: 176426 Complete
product_id: 102945 Scraping
product_id: 102945 Complete
product_id: 168473 Scraping
product_id: 168473 Complete
product_id: 41479 Scraping
product_id: 41479 Complete
product_id: 30280 Scraping
product_id: 30280 Complete
product_id: 2637 Scraping
product_id: 2637 Complete
product_id: 1109 Scraping
product_id: 1109 Complete
product_id: 115983 Scraping
product_id: 115983 Complete
product_id: 44782 Scraping
product_id: 44782 Complete
product_id: 143455 Scraping
product_id: 143455 Complete
product_id: 150608 Scraping
product_id: 150608 Complete
product_id: 19498 Scraping
product_id: 19498 Complete
product_id: 135950 Scraping
product_id: 135950 Complete
product_id: 44393 Scraping
product_id: 44393 Complete
product_id: 65920 Scraping
product_id: 65920 Complete
https://www.makeupalley.com/product/browse.asp/page=3/pagesize=15/Categ

product_id: 67549 Scraping
product_id: 67549 Complete
product_id: 69351 Scraping
product_id: 69351 Complete
https://www.makeupalley.com/product/browse.asp/page=1/pagesize=15/CategoryId=507/topten=reviewed/AgeRange=0/
product_id: 153046 Scraping
product_id: 153046 Complete
product_id: 105706 Scraping
product_id: 105706 Complete
product_id: 147152 Scraping
product_id: 147152 Complete
product_id: 153812 Scraping
product_id: 153812 Complete
product_id: 162230 Scraping
product_id: 162230 Complete
product_id: 133878 Scraping
Skipping a page this didn't work
product_id: 133878 Complete
product_id: 152906 Scraping
Skipping a page this didn't work
product_id: 152906 Complete
product_id: 117298 Scraping
product_id: 117298 Complete
product_id: 149584 Scraping
product_id: 149584 Complete
product_id: 158019 Scraping
product_id: 158019 Complete
product_id: 158397 Scraping
product_id: 158397 Complete
product_id: 158134 Scraping
product_id: 158134 Complete
product_id: 157566 Scraping
product_id: 15756

In [10]:
products_df = pd.DataFrame(product_info, columns = ["Brand", "Product", "Type", "Avg Rating", "Number of Reviews", "Percentage Would Buy Again"])

In [11]:
products_df

,Brand,Product,Type,Avg Rating,Number of Reviews,Percentage Would Buy Again
0,MAC Cosmetics,Studio Fix Powder Plus Foundation,Powder,3.7,3938,0.66
1,bareMinerals,Bare Minerals ALL,Powder,2.9,3074,0.44
2,bareMinerals,"Bare Minerals in 1.2, fairly light",Powder,3.5,1152,0.59
3,L'Oreal Paris,Bare Naturale Powdered Mineral Foundation,Powder,3.5,1092,0.62
4,Physicians Formula,Mineral Wear Talc-Free Mineral Loose Powder,Powder,3.6,800,0.65
5,bareMinerals,Matte Foundation spf 15,Powder,3.4,762,0.59
6,Jane Iredale,Pure Pressed Base,Powder,4.0,664,0.72
7,Maybelline New York,Mineral Power Powder Foundation,Powder,3.2,639,0.55
8,Laura Mercier,Mineral Powder SPF 15,Powder,4.0,592,0.71
9,Everyday Minerals,Semi-Matte Foundation,Powder,3.8,550,0.68


In [14]:
reviews_df = pd.DataFrame(reviews, columns = ["Type", "Product", "Rating", "Username", "Date", "Age", "Skin", "Hair", "Eyes", "Review"])

In [21]:
reviews_df

,Type,Product,Rating,Username,Date,Age,Skin,Hair,Eyes,Review
0,Powder,Studio Fix Powder Plus Foundation,4,Marlen86,on 11/25/2018 4:35:00 PM,30-35,"Oily, Fair-Medium, Not Sure","Brown, Other, Other",Brown,"Nice pigmented powder, especially to cover ski..."
1,Powder,Studio Fix Powder Plus Foundation,5,babypuggies,on 10/31/2018 9:42:00 AM,19-24,"Combination, Medium, Warm","Blond, Straight, Medium",Brown,This has become my go-to foundation now! My oi...
2,Powder,Studio Fix Powder Plus Foundation,4,ChristopherCupcakes,on 10/22/2018 1:28:00 PM,44-55,"Combination, Medium, Neutral","Other, Straight, Medium",Green,This is an amazing powder foundation that leav...
3,Powder,Studio Fix Powder Plus Foundation,4,congolesedoll,on 10/9/2018 4:39:00 PM,19-24,"Very Oily, Dark, Not Sure","Blond, Other, Other",Brown,Amazing powder that keeps you MATTE and keeps ...
4,Powder,Studio Fix Powder Plus Foundation,5,manowaris,on 10/9/2018 2:58:00 PM,19-24,"Acne-prone, Fair, Neutral","Brown, Wavy, Coarse",Blue,I bought this powder a lot. I love it. It has ...
5,Powder,Studio Fix Powder Plus Foundation,4,ccarlino,on 10/5/2018 12:28:00 AM,56 & Over,"Dry, Fair-Medium, Cool","Silver, Other, Medium",Hazel,I've been using MAC's Studio Fix Powder founda...
6,Powder,Studio Fix Powder Plus Foundation,2,1319tle,on 10/3/2018 11:31:00 PM,19-24,"Dry, Fair-Medium, Cool","Black, Wavy, Fine",Black,i have dry skin and this makes my skin look wo...
7,Powder,Studio Fix Powder Plus Foundation,4,jts123,on 9/22/2018 6:14:00 PM,19-24,"Oily, Fair, Neutral","Blond, Wavy, Medium",Blue,"TLDR; Great, buildable, flawless coverage but ..."
8,Powder,Studio Fix Powder Plus Foundation,5,Stoviesgirl,on 9/10/2018 11:27:00 AM,44-55,"Dry, Fair, Not Sure","Brunette, Other, Other",Blue,I love all things MAC. Every product I've trie...
9,Powder,Studio Fix Powder Plus Foundation,4,auraborialis,on 9/10/2018 10:38:00 AM,36-43,"Combination, Fair-Medium, Warm","Brown, Curly, Medium",Brown,I hate that I love this foundation. I have NC2...


In [27]:
import numpy as np

In [33]:
reviews_df["Review"] = reviews_df['Review'].map(lambda x: x.encode('unicode-escape').decode('utf-8'))

In [34]:
reviews_df.to_csv("reviews.csv")

In [35]:
products_df.to_csv("products.csv")